In [1]:
import os 
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings, OpenAIEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [2]:
from urllib.request import urlretrieve, build_opener, install_opener

In [3]:
opener = build_opener()
opener.addheaders = [("User-Agent", "Mozilla/5.0")]
install_opener(opener)

In [4]:
url = "http://assets.fiba.basketball/image/upload/documents-corporate-fiba-official-rules-2024-v10a.pdf"
os.makedirs("FIBA", exist_ok=True)

In [5]:
file_path = os.path.join("FIBA", url.rpartition("/")[2])

In [6]:
file_path

'FIBA\\documents-corporate-fiba-official-rules-2024-v10a.pdf'

In [7]:
urlretrieve(url, file_path)

('FIBA\\documents-corporate-fiba-official-rules-2024-v10a.pdf',
 <http.client.HTTPMessage at 0x2558e438620>)

In [8]:
loader = PyPDFLoader(r"C:\Users\tmakh\OneDrive\Desktop\AI_Lara_Wehbe\python\LLMs\Final_Project\FIBA\documents-corporate-fiba-official-rules-2024-v10a.pdf")
doc_before_split = loader.load()

In [9]:
print(doc_before_split[10].page_content)                    

October 2024 OFFICIAL BASKETBALL RULES 2024 Page 11 of 105 
Article 3 Equipment The following equipment shall be required: • Backstop units, consisting of: ▬ Backboards ▬ Baskets comprising (pressure release) rings and nets ▬ Backboard support structures including padding. • Basketballs • Game clock • Scoreboard • Shot clock • Stopwatch or suitable (visible) device (not the game clock) for timing time-outs • 2 separate, distinctly different and loud signals, one of each for the ▬ shot clock operator, ▬ timer. • Scoresheet • Player / head coach foul markers • Team foul markers • Alternating possession arrow • Floor • Court • Adequate lighting. For a more detailed description of basketball equipment, see the Appendix on Basket-ball Equipment.


In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 60
)

In [11]:
doc_after_split = text_splitter.split_documents(doc_before_split)

In [12]:
doc_after_split[50].page_content

'not on the court but entitled to play. • An excluded player when having committed 5 fouls and is no longer entitled to play.  4.1.4 During any interval of play, all team members entitled to play are considered as players. 4.2 Rule 4.2.1 Each team shall consist of: • No more than 12 team members entitled to play, including a captain. • A head coach. • A maximum of 8 accompanying delegation members, including assistant coaches who may sit on the team bench. In case a team has assistant coaches, the first assistant coach shall be entered on the scoresheet. 4.2.2 During playing time 5 team'

In [13]:
hugging_face_embeddings = HuggingFaceBgeEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs = {"device": "cpu"},
    encode_kwargs = {"normalize_embeddings" : True}
)

C:\Users\tmakh\AppData\Local\Temp\ipykernel_16296\3179441968.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hugging_face_embeddings = HuggingFaceBgeEmbeddings(


In [14]:
from langchain_community.vectorstores import Chroma

vectorestore = Chroma.from_documents(doc_after_split, hugging_face_embeddings)

In [15]:
query = "What are the rules of dribling"

In [16]:
relevant_documents = vectorestore.similarity_search(query)

In [17]:
print(relevant_documents[1].page_content)

Article 24 Dribbling 24.1 Definition 24.1.1 A dribble is the movement of a live ball caused by a player in control of that ball who throws, taps, rolls or bounces the ball on the court.  24.1.2 A dribble starts when a player, having gained control of a live ball on the court throws, taps, rolls or bounces it on the court and touches it again before it touches another player. During a dribble the player may not place any part of the hand under the ball and carry it from one point to another or bring the ball to a pause and then continue to dribble. During a dribble the ball may be thrown into


In [18]:
retriever = vectorestore.as_retriever(
    search_type= "similarity",
    search_kwargs = {"k": 3}
)

In [19]:
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")


In [20]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1,
    max_tokens=500,
    api_key=api_key
)

In [21]:
output = llm.invoke(query)
print(output)

content="Dribbling is a fundamental skill in basketball, soccer, and other sports that involves controlling a ball with one's hands or feet. Here are the basic rules and techniques for dribbling:\n\n**General Rules:**\n\n1. **Control the ball**: The primary goal of dribbling is to maintain control of the ball while moving it in different directions.\n2. **Use the correct body position**: Keep your head up, shoulders relaxed, and knees slightly bent to maintain balance and stability.\n3. **Keep the ball close**: Keep the ball close to your body to maintain control and prevent it from being stolen.\n4. **Use different parts of the body**: Use different parts of your body, such as your hands, feet, or head, to control the ball.\n5. **Dribble with both hands**: In basketball, dribbling with both hands is essential to maintain control and create scoring opportunities.\n\n**Soccer Dribbling Rules:**\n\n1. **Use the inside, outside, and sole of the foot**: In soccer, dribblers use the inside,

In [ ]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the answer in the documents. "
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)


In [23]:
retrievalQA = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [24]:
result = retrievalQA.invoke({"query" : query})
print(result)

{'query': 'What are the rules of dribling', 'result': 'According to Article 24 of the rules, a dribble is defined as the movement of a live ball caused by a player in control of that ball who throws, taps, rolls, or bounces the ball on the court. A dribble starts when a player gains control of a live ball and touches it again before it touches another player. During a dribble, the player may not place any part of the hand under the ball and carry it from one point to another or bring the ball to a pause and then continue to dribble. A dribble ends when the player touches the ball with both hands simultaneously or permits the ball to come to rest in one or both hands.', 'source_documents': [Document(metadata={'creator': 'Word', 'moddate': '2024-10-02T16:21:04+03:00', 'creationdate': '2024-10-02T13:18:52+00:00', 'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'page': 29, 'source': 'C:\\Users\\tmakh\\OneDrive\\Desktop\\AI_Lara_Wehbe\\python\\LLMs\\Final_Project\\FIBA\\

In [25]:
print(result.keys())

dict_keys(['query', 'result', 'source_documents'])


In [26]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

There are 3 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: C:\Users\tmakh\OneDrive\Desktop\AI_Lara_Wehbe\python\LLMs\Final_Project\FIBA\documents-corporate-fiba-official-rules-2024-v10a.pdf, Page: 29
Content: • Deflecting a pass and gaining control of the ball. • Tossing the ball from hand to hand and allowing it to come to rest in one or both hands before touching the court, provided that no travelling violation is committed. • Throwing the ball against the backboard and regaining the control of the ball. 24.2 Rule A player shall not dribble for a second time after the first dribble has ended unless between the 2 dribbles: • The control of a live ball on the court is lost because of a shot for a goal. • The ball is touched by an opponent. • On a pass or fumble the ball touches or is touched by
----------------------------------------------------

In [27]:
def answer_question(history, query):
    result = retrievalQA.invoke({"query":query})
    answer = result["result"]
    sources = ""
    for i, doc in enumerate(result['source_documents']):
        sources += f"Document #{i+1}:\nSource: {doc.metadata['source']}, Page: {doc.metadata['page']}\n{doc.page_content}\n\n"
    history.append((query, answer))
    return history, ""

def clear_chat():
    return [], ""

In [28]:
import gradio as gr
def gradio_app():
    with gr.Blocks() as app:
        gr.Markdown("# 🏀 FIBA rules")
        gr.Markdown("Ask a question about FIBA rules")
        chatbot = gr.Chatbot(label="Chat Interference")
        user_input = gr.Textbox(label="Your message",
                                placeholder="Ask your question...")
        send_button = gr.Button("Send")
        clear_button = gr.Button("Clear")

        send_button.click(
            fn=answer_question,
            inputs=[chatbot, user_input],
            outputs=[chatbot, user_input]
        )
        clear_button.click(
            fn=clear_chat,
            inputs=[],
            outputs=[chatbot, user_input]
        )
    return app

In [29]:
app = gradio_app()
app.launch()

C:\Users\tmakh\AppData\Local\Temp\ipykernel_16296\243338029.py:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat Interference")


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
